In [27]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/processed/features_with_clusters.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])

df.head()


,user_id,timestamp,amount,time_since_last_txn,txn_count_1h,amount_zscore,device_change_rolling,geo_distance_rolling,risk_momentum,strategy_change,channel,location,is_fraud,cluster
0,1,2024-01-03 03:48:02,5.15,NaN,1.0,NaN,0.0,4.250000,NaN,0.0,card,rural,0,0
1,1,2024-01-04 07:10:57,37.05,98575.0,1.0,0.677090,0.0,5.190000,NaN,0.0,netbanking,semi-urban,0,0
2,1,2024-01-05 18:50:51,137.87,128394.0,1.0,1.107692,0.0,5.306667,0.371602,0.0,card,rural,0,0
3,1,2024-01-05 22:04:41,47.32,11630.0,1.0,-0.164491,0.0,5.672500,-0.820200,0.0,netbanking,rural,0,0
4,1,2024-01-06 06:48:59,93.75,31458.0,1.0,0.557185,0.0,5.952500,0.252694,0.0,card,semi-urban,0,0


In [28]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")
sys.path.insert(0, PROJECT_ROOT)

print("Project root added to path:", PROJECT_ROOT)


Project root added to path: c:\Users\jadha\OneDrive\Desktop\project2


In [29]:
cluster_risk = (
    df.groupby("cluster")["is_fraud"]
      .mean()
      .to_dict()
)

cluster_risk


{-1: 0.2763121355179117,
 0: 0.010827703074451462,
 1: 0.013201320132013201,
 2: 0.0196078431372549}

In [30]:
df["cluster_risk"] = df["cluster"].map(cluster_risk)

df["risk_score"] = (
    0.35 * df["amount_zscore"].abs() +
    0.25 * df["risk_momentum"].clip(lower=0) +
    0.20 * df["device_change_rolling"] +
    0.20 * df["cluster_risk"]
)

df["risk_score"] = df["risk_score"].clip(0, 1)


In [31]:
user_risk = (
    df.groupby("user_id")["risk_score"]
      .mean()
      .reset_index(name="avg_user_risk")
)

user_risk.head()


,user_id,avg_user_risk
0,1,0.325552
1,2,0.359781
2,3,0.393627
3,4,0.322896
4,5,0.318908


In [32]:
from src.explainability import explain_transaction

df["risk_explanation"] = df.apply(explain_transaction, axis=1)

df[["risk_score", "risk_explanation"]].head()


,risk_score,risk_explanation
0,NaN,No significant risk signals detected
1,NaN,No significant risk signals detected
2,0.482758,Escalating risk pattern
3,0.059738,No significant risk signals detected
4,0.260354,Escalating risk pattern


In [33]:
# SAVE FINAL TABLE WITH EXPLANATIONS
df.to_csv("../data/processed/final_risk_table.csv", index=False)

print("final_risk_table.csv saved WITH risk_explanation")
print(df.columns.tolist())


final_risk_table.csv saved WITH risk_explanation
['user_id', 'timestamp', 'amount', 'time_since_last_txn', 'txn_count_1h', 'amount_zscore', 'device_change_rolling', 'geo_distance_rolling', 'risk_momentum', 'strategy_change', 'channel', 'location', 'is_fraud', 'cluster', 'cluster_risk', 'risk_score', 'risk_explanation']
